In [5]:
import json
import git
from git import Repo


f = open("sstubs.json")
sstubs = json.load(f)

repo_name_list1 = ['apache.camel']
repo_name_list2 = ['hazelcast.hazelcast', 'Alluxio.alluxio']
repo_name_list3 = ['libgdx.libgdx', 'wildfly.wildfly', 'checkstyle.checkstyle',]
repo_name_list4 = ['google.closure-compiler', 'spring-projects.spring-boot','apache.flink','facebook.presto',]
repo_name_list5 = [
    'netty.netty',
    'openhab.openhab',
    'apache.hbase',
    'Atmosphere.atmosphere',
    'openmrs.openmrs-core',
    
    'druid-io.druid',
    'gephi.gephi',
]

url_list1 = [
    'https://github.com/apache/camel'
]
url_list2 = [
    'https://github.com/hazelcast/hazelcast',
    'https://github.com/Alluxio/alluxio'
]
url_list3 = [
    'https://github.com/libgdx/libgdx',
    'https://github.com/wildfly/wildfly',
    'https://github.com/checkstyle/checkstyle',
]
url_list4 = [
    'https://github.com/google/closure-compiler',
    'https://github.com/spring-projects/spring-boot',
    'https://github.com/apache/flink',
    'https://github.com/facebook/presto',
]
url_list5 = [
    'https://github.com/netty/netty',
    'https://github.com/openhab/openhab',
    'https://github.com/apache/hbase',
    'https://github.com/Atmosphere/atmosphere',
    'https://github.com/openmrs/openmrs-core',
    
    'https://github.com/druid-io/druid',
    'https://github.com/gephi/gephi',
]

repo_name_list = repo_name_list1 + repo_name_list2 + repo_name_list3 + repo_name_list4 #+ repo_name_list5
# get the fixing commits belong to the repo_list
# key: bug fixing commit's id
# value:
#  bugFilePath: the path to the fixed file and line_nums (dic, can be multiple)
#  fixCommitParentSHA1, parent_commit_id

filter_commits = {}

for bug in sstubs:
    if bug['projectName'] not in repo_name_list:
        continue
    cmt_id = bug["fixCommitSHA1"]
    parent_commit = bug["fixCommitParentSHA1"]
    bug_file_path = bug["bugFilePath"]
    bug_line_num = bug["bugLineNum"]
    project_name = bug["projectName"]
    
    if cmt_id in filter_commits:
        # old commit, check if new file path
        if bug_file_path in filter_commits[cmt_id]["bugFilePath"]:
            # old file path, check if new line num
            if bug_line_num in filter_commits[cmt_id]["bugFilePath"][bug_file_path]:
                # bug already exist
                continue
            else:
                # new bug in old file path
                filter_commits[cmt_id]["bugFilePath"][bug_file_path].add(bug_line_num)
        else:
            # new file path
            filter_commits[cmt_id]["bugFilePath"][bug_file_path] = {bug_line_num}
    else:
        # new commit
        filter_commits[cmt_id] = {
            'fixCommitParentSHA1': parent_commit,
            'bugFilePath': {bug_file_path: {bug_line_num}},
            'projectName': project_name
        }

print("the size of filtered bug fixing commit is {}".format(len(filter_commits)))
# print a item in dic
for x, y in filter_commits.items():
    print(x)
    print(y)
    break

the size of filtered bug fixing commit is 2149
92b7f309d809fbb4e74a0677f62b8fe39ff207db
{'fixCommitParentSHA1': '1155b10c8a1089bdeaebf752311991ce015b9d00', 'bugFilePath': {'examples/src/main/java/alluxio/examples/UnderFileSystemCommonOperations.java': {179}}, 'projectName': 'Alluxio.alluxio'}


In [10]:

fix_introduce_pairs = {}

for key, value in filter_commits.items():
    repo_name = value["projectName"]
    repo_path = "/Users/lixinkai/Desktop/repos/" + repo_name.split(".")[1]
    repo = Repo(repo_path)
    repo.git.checkout('master')

    parent_commit = value["fixCommitParentSHA1"]
    for path_name, line_nums in value["bugFilePath"].items():
        for line_num in line_nums:
            line = 1
            for commit, lines in repo.blame(parent_commit, path_name):
                if line <= line_num < (line + len(lines)):
                    if key in fix_introduce_pairs:
                        if commit not in fix_introduce_pairs[key]['bug_inducing_commit_ids']:
                            fix_introduce_pairs[key]['bug_inducing_commit_ids'].add(commit.hexsha)
                    else:
                        fix_introduce_pairs[key] = {
                            'bug_inducing_commit_ids': {commit.hexsha},
                            'project_name': repo_name
                        }
                line += len(lines)
    

print("the size of fixing-inducing pair is {}".format(len(fix_introduce_pairs)))
# print a item in dic
for x, y in fix_introduce_pairs.items():
    print(x)
    print(y)
    break


the size of fixing-inducing pair is 2149
92b7f309d809fbb4e74a0677f62b8fe39ff207db
{'bug_inducing_commit_ids': {'e2110b5a6672c889b149b8d276f4374842709ba2'}, 'project_name': 'Alluxio.alluxio'}


In [11]:
data = []

for key, value in fix_introduce_pairs.items():
    project_name = value['project_name']
    commit_ids = list(value['bug_inducing_commit_ids'])
    data.append({
        'bug_fixing_commit_id': key,
        'bug_inducing_commit_ids': commit_ids,
        'project_name': project_name
    })
    
with open('fix_introduce_pairs_v2.json', 'w') as outfile:
    json.dump(data, outfile)


In [8]:
## do not need for now

bug_fixing_commits = []
for key, value in filter_commits.items():
    bug_inducing_commit_ids = list(fix_introduce_pairs[key])
    locations = {}
    bugFilePath = value["bugFilePath"]
    for path_name, line_nums in bugFilePath.items():
        locations[path_name] = {
            'count': len(list(line_nums)),
            'locations': list(line_nums)
        }
    bug_fixing_commits.append({
        'bug_fixing_commit_id': key,
        'fixCommitParentSHA1': value["fixCommitParentSHA1"],
        'bug_inducing_commit_ids': bug_inducing_commit_ids,
        'bugFilePath': locations
    })
    
with open('fix_introduce_pairs_v2.json', 'w') as outfile:
    json.dump(bug_fixing_commits, outfile)

In [5]:
ln = 156
tlc = 1
commits = set()
for commit, lines in repo.blame('69fcae6b44cb9443bd970f89f7215258d6092bcc', 'spring-boot-actuator/src/main/java/org/springframework/boot/actuate/autoconfigure/HealthIndicatorAutoConfiguration.java'):
    if tlc <= ln < (tlc + len(lines)):
        commits.add(commit.hexsha)
        print(commit)
        print(lines)
    tlc += len(lines)
print(commits)

b569918db1c33847ce1d1464d7d78f692f289e99
['\t@ConditionalOnEnablednHealthIndicator("db")']
{'b569918db1c33847ce1d1464d7d78f692f289e99'}
